# <center> TensorFlow </center>

![](tflogo.png)

*<p style='text-align:right'>By TensorFlow - vectors combined, edited - Begoon [Apache License 2.0] via Wikimedia Commons</p>*

- one of the most popular tools for deep learning
- released in 2015
- "framework for numerical computation based on dataflow graphs." [2] [Hope et al. (2017)]
- tensors - multidimensional arrays
- built in C++, used through higer level languages (Python) and libraries (keras, tf-slim)

![](sqcube.png)

![](greyrgb.png)

- image tensor flows through the computational graph

### Computational graph:

![](compgraph.png)

- composed of nodes and edges

*<p style='text-align:right'>Computational Graph. By Sebastian Raschka via Github</p>*

### Advantages and Disadvantages (a few):

- open source and continously developed
- good visualization tools 
- scalability and portability
- steeper learning curve to become good at it 
- implies solid knowledge of math
- http://tensorflow.org

### Types of Operations:

- Add, Sub, Div, Mul, Log, Exp, etc. (element wise math ops.)
- on arrays and matrices: Constant, Shape, Split, Concat, Slice, MatMul, etc.
- Sigmoid, ReLU, Convolution2D, SoftMax, etc.
- and more

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a = tf.constant('Hello')
b = tf.constant(' World!')
c = a+b

In [3]:
print(c)

Tensor("add:0", shape=(), dtype=string)


In [4]:
with tf.Session() as sess: 
    runop = sess.run(c)
    
print(runop)

b'Hello World!'


In [5]:
c = tf.add(a,b)

with tf.Session() as sess:
    runop = sess.run(c)
    
print(runop)

b'Hello World!'


In [6]:
mat_a = tf.constant(np.arange(1,12, dtype=np.int32), shape=[2,2,3])
mat_b = tf.constant(np.arange(12,24, dtype=np.int32), shape=[2,3,2])
mul_c = tf.matmul(mat_a, mat_b)

with tf.Session() as sess:
    runop = sess.run(mul_c)
    
print(runop)

[[[ 88  94]
  [214 229]]

 [[484 508]
  [642 674]]]


### A typical TensorFlow project:

- (1). constructing the graph: tf.constant, tf.placeholder, tf.variable, tf.nn.sigmoid, tf.matmul, etc.
- (2). executing the graph (within a session)
- use tensorboard for visualization.

### Placeholders:

- allow for reusability
- can be populated with different data every time you run the graph (feed_dict)

In [7]:
#with tf.Session() as sess:
#    sess.run(ops)
#    sess.close()

In [8]:
x = tf.placeholder(tf.float32, name='X', shape=(4,9))
w = tf.placeholder(tf.float32, name='W', shape=(9,1))
b = tf.fill((4,1), -1., name='bias')
y = tf.matmul(x,w)+b
s = tf.reduce_max(y)

In [9]:
x_data = np.random.randn(4,9)
w_data = np.random.randn(9,1)

with tf.Session() as sess:
    out_p = sess.run(s, feed_dict={x:x_data, w:w_data})
    
print('Output: {}'.format(out_p))

Output: 2.9670121669769287


### Optimization

- purpose of a typical project: learn associations between features and labels.

### <center> $f(x_{i}) = w^{T}x_{i} + b$ </center> 
### <center> $y_{i} = f(x_{i}) + \varepsilon_{i}$ </center>

- **Loss Function**: how far the model is from the truth.
- train the model to minimize loss (adjusting the parameters).
- **MSE (Mean Squared Error)** - averages square distances (y_true, y_pred):

## <center> $L(y, \hat y) = \frac{1}{n}\ ∑_{i=1}^{n}(y_{i}-\hat y_{i})^{2}$ </center>

- **Cross Entropy** - similarity between distributions:

## <center> $H(p,q)=-∑_{x}p(x)logq(x)$ </center>

In [10]:
# X, Y, W, b

x = tf.placeholder(tf.float32, shape=[None, 3])
y_true = tf.placeholder(tf.float32, shape=None)
w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
b = tf.Variable(0, dtype=tf.float32, name='b')

# Output

y_pred = tf.matmul(w, tf.transpose(x))+b

In [11]:
# MSE (Mean Squared Error)

loss = tf.reduce_mean(tf.square(y_true-y_pred))

# Cross Entropy

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
loss = tf.reduce_mean(loss)

### Linear Regression 

- Batch Gradient Descent: entire dataset.
- Stochastic Gradient Descent (SGD): mini-batches. 
- In TensorFlow:

<pre>optimizer = tf.train.GradientDescentOptimizer(learning_rate)</pre>
<pre>train = optimizer.minimize(loss)</pre>


In [12]:
import numpy as np

x_data = np.random.randn(2000,3)
w_real = [0.4, 0.6, 0.2]
b_real = -0.3 
noise = np.random.randn(1,2000)*0.1 
y_data = np.matmul(w_real, x_data.T)+b_real+noise

In [13]:
num_iters = 10 
g = tf.Graph()
wb = []

with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3])
    y_true = tf.placeholder(tf.float32, shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
        b = tf.Variable(0, dtype=tf.float32, name='b')
        y_pred = tf.matmul(w, tf.transpose(x))+b
        
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true-y_pred))
        
    with tf.name_scope('training') as scope:
        lr = 0.5
        optimizer = tf.train.GradientDescentOptimizer(lr)
        train = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for step in range(num_iters):
            sess.run(train, {x:x_data, y_true:y_data})
            if(step%5==0):
                print(step, sess.run([w,b]))
                wb.append(sess.run([w,b]))
                
        print(10, sess.run([w,b]))

0 [array([[ 0.4324733 ,  0.62102544,  0.21289678]], dtype=float32), -0.26101521]
5 [array([[ 0.39815262,  0.59838271,  0.20108758]], dtype=float32), -0.30086589]
10 [array([[ 0.39815292,  0.59838313,  0.20108768]], dtype=float32), -0.30086562]


### Logistic Regression 

In [14]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

x_data = np.random.randn(20000,3)
w_real = [0.4, 0.6, 0.2]
b_real = -0.3
wb = np.matmul(w_real,x_data.T)+b_real

y_data_bef_noise = sigmoid(wb)
y_data = np.random.binomial(1, y_data_bef_noise)

In [15]:
num_iters = 50
g = tf.Graph()
wb = []

with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3])
    y_true = tf.placeholder(tf.float32, shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
        b = tf.Variable(0, dtype=tf.float32, name='b')
        y_pred = tf.matmul(w, tf.transpose(x))+b
        
    with tf.name_scope('loss') as scope:
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
        loss = tf.reduce_mean(loss)
        
    with tf.name_scope('training') as scope:
        lr = 0.5
        optimizer = tf.train.GradientDescentOptimizer(lr)
        train = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        for step in range(num_iters):
            sess.run(train, {x:x_data, y_true:y_data})
            if(step%5==0):
                print(step, sess.run([w,b]))
                wb.append(sess.run([w,b]))
                
        print(50, sess.run([w,b]))

0 [array([[ 0.04192033,  0.06381649,  0.02010243]], dtype=float32), -0.032525264]
5 [array([[ 0.18624175,  0.28245297,  0.08911894]], dtype=float32), -0.1444058]
10 [array([[ 0.26498973,  0.40079242,  0.12666152]], dtype=float32), -0.20510893]
15 [array([[ 0.31052727,  0.46876729,  0.14835776]], dtype=float32), -0.23986366]
20 [array([[ 0.33786425,  0.50934786,  0.16139235]], dtype=float32), -0.260488]
25 [array([[ 0.35464621,  0.53414452,  0.16940555]], dtype=float32), -0.27300319]
30 [array([[ 0.36508736,  0.54951191,  0.1743995 ]], dtype=float32), -0.28070512]
35 [array([[ 0.3716363 ,  0.55911851,  0.17753723]], dtype=float32), -0.28548792]
40 [array([[ 0.37576416,  0.56515634,  0.17951828]], dtype=float32), -0.28847569]
45 [array([[ 0.37837368,  0.56896406,  0.18077263]], dtype=float32), -0.29034957]
50 [array([[ 0.3797532 ,  0.57097304,  0.1814366 ]], dtype=float32), -0.29133382]


## Softmax Regression with MNIST



<center>![](mnist.png)</center>
*<p style='text-align:right'>By Josef Steppan (Own work), via Wikimedia Commons</p>*

In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [17]:
datadir = '/data'
num_iters = 1000
minibatch_size = 100

In [18]:
data = input_data.read_data_sets(datadir, one_hot=True)
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

correct_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(num_iters):
        batch_xs, batch_ys = data.train.next_batch(minibatch_size)
        sess.run(optimizer, feed_dict={x:batch_xs, y_true:batch_ys})
        
    testing = sess.run(accuracy, feed_dict={x:data.test.images, y_true:data.test.labels})
    
print('Accuracy: {:.4}%'.format(testing*100))

Extracting /data\train-images-idx3-ubyte.gz
Extracting /data\train-labels-idx1-ubyte.gz
Extracting /data\t10k-images-idx3-ubyte.gz
Extracting /data\t10k-labels-idx1-ubyte.gz
Accuracy: 91.66%


## Convolutional Neural Networks

<center>![](cnn.png)</center>
*<p style='text-align:right'>By Aphex34 (Own work) [CC BY-SA 4.0], via Wikimedia Commons</p>*

### Good resources:

- CS231n from Stanford University
- Convolutional Neural Networks (Course 4) by Prof. Andrew Ng - Deep Learning Specialization


#### Convolution in TensorFlow:

<center><pre>tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')</pre></center>

- x.shape is [None, 28, 28, 1]
- W.shape is [5, 5, 1, 32]
- strides - how the filter moves across the image
- 'SAME' padding - output size = input size.

### Activations

- fully connected, covolutional layers - non-linear activations (sigmoid, tanh, relu).

### Pooling 

- reduces the size of the data
- reduces no. of hyper-params.

<center><pre>tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')</pre></center>

### Dropout

- for regularization
- turns off neurons or units in a given layer.

<center><pre>tf.nn.dropout(layer, keep_prob=keep_prob)</pre></center>

### Helper Functions

In [19]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W)+b)

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W)+b 

### The Model (A CNN Approach)

In [20]:
x = tf.placeholder(tf.float32, shape=[None, 784]) 
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

conv1 = conv_layer(x_image, shape=[5,5,1,32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5,5,32,64])
conv2_pool = max_pool_2x2(conv2)
conv2_flat = tf.reshape(conv2_pool, [-1,7*7*64])

full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))
keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)
y_conv = full_layer(full1_drop, 10)

In [25]:
mnist = input_data.read_data_sets(datadir, one_hot=True)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting /data\train-images-idx3-ubyte.gz
Extracting /data\train-labels-idx1-ubyte.gz
Extracting /data\t10k-images-idx3-ubyte.gz
Extracting /data\t10k-labels-idx1-ubyte.gz


In [28]:
steps=180 
                                                             
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    
    for i in range(steps):
        
        batch = mnist.train.next_batch(50) 
        
        if i % 10 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0],y_: batch[1],keep_prob: 1.0}) 
            print ("step {}, training accuracy {}".format(i, train_accuracy)) 
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1],keep_prob: 0.5}) 

        
    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10, 1000, 10) 
    test_accuracy = np.mean([sess.run(accuracy,feed_dict={x:X[i], y_:Y[i],keep_prob:1.0}) for i in range(10)])

    
print ("test accuracy: {}".format(test_accuracy))

step 0, training accuracy 0.03999999910593033
step 10, training accuracy 0.18000000715255737
step 20, training accuracy 0.5600000023841858
step 30, training accuracy 0.47999998927116394
step 40, training accuracy 0.7400000095367432
step 50, training accuracy 0.6800000071525574
step 60, training accuracy 0.8600000143051147
step 70, training accuracy 0.8399999737739563
step 80, training accuracy 0.7599999904632568
step 90, training accuracy 0.7799999713897705
step 100, training accuracy 0.8799999952316284
step 110, training accuracy 0.800000011920929
step 120, training accuracy 0.8799999952316284
step 130, training accuracy 0.8999999761581421
step 140, training accuracy 0.8999999761581421
step 150, training accuracy 0.9200000166893005
step 160, training accuracy 0.8799999952316284
step 170, training accuracy 0.9399999976158142
test accuracy: 0.9029000401496887
